# Modelo de Transporte- PL

In [1]:
# El problema de transporte, en su forma más básica consiste en determinar 
# cuanto debo enviar desde uno o más origenes a n destinos, con el objetivo de satisfacer la demanda
# de los destinos sujeto a restricciones de disponibilidad en los origenes. Enviar una cantidad desde un origen a un destino, cuesta dinero, tarda tiempo, etc.
# Lo que se desea en este ejemplo es minimizar el costo del transporte, y se cuenta con 3 centros de Distribución (CDs) y 4 destinos, que corresponden a locales de venta.

### Importación de Librerías y Módulos

In [2]:
import pandas as pd
import numpy as np
import pulp

### Parámetros del Modelo

In [3]:
# 4 tiendas son abastecidas desde 3 centros de distribución. Los requerimientos de las tiendas 
# se definen en las siguientes variables

In [4]:
locales = ["a","b","c","d"]
requerimientos_local =  {"a":80, "b":65, "c":70,"d":85} # el local a requiere 80 unidades, el b 65 y así.

In [5]:
sum(requerimientos_local.values())

300

In [6]:
# Respecto de los CDs, los datos de disponibilidad y Costo de trasnporte desde cada CD a cada local,
# se encuentran en el csv adjunto.

In [8]:
datos = pd.read_csv('datos_transporte.csv', sep = ';') # ruta del archivo, separado por ;

In [9]:
datos

,Centros de Distribucion,Disponibilidad,a,b,c,d
0,1,75,46,51,65,86
1,2,125,35,41,69,79
2,3,100,99,68,38,68


In [10]:
datos["Disponibilidad"].sum()

300

In [11]:
#### Se definen los conjuntos y parámetros faltantes

In [12]:
cds = list(datos["Centros de Distribucion"])
cds

[1, 2, 3]

In [13]:
disponibilidad_cd = {
    cd: datos.loc[datos["Centros de Distribucion"] == cd, "Disponibilidad"].iloc[0]
    for cd in cds
}
disponibilidad_cd

{1: 75, 2: 125, 3: 100}

In [14]:
#### matriz de costos

In [15]:
costos = {
    (cd, local): datos.loc[datos["Centros de Distribucion"] == cd, local].iloc[0]
    for cd in cds for local in locales
}
costos

{(1, 'a'): 46,
 (1, 'b'): 51,
 (1, 'c'): 65,
 (1, 'd'): 86,
 (2, 'a'): 35,
 (2, 'b'): 41,
 (2, 'c'): 69,
 (2, 'd'): 79,
 (3, 'a'): 99,
 (3, 'b'): 68,
 (3, 'c'): 38,
 (3, 'd'): 68}

### Formulación del Problema

In [16]:
def get_problem(locales, requerimientos_local, cds, disponibilidad_cd, costos):
    # instanciar el problema
    problem = pulp.LpProblem("Minimizar_Costos_Transporte", pulp.LpMinimize)

    # definición de variables

    # cantidad a enviar desde el centro de distribucion c al local l
    X = pulp.LpVariable.dicts("X", ((c, l) for c in cds for l in locales), lowBound=0, cat='Continuous')
    
    # definición de la función objetivo: minimizar los costos
    problem += pulp.lpSum(costos[(c,l)] * X[(c,l)] for c in cds for l in locales )

    # definicón de las restricciones

    # lo que se envía de cada CD debe ser menor o igual a la disponibilidad en ese CD 
    for c in cds:
        problem += pulp.lpSum(X[(c,l)] for l in locales) <= disponibilidad_cd[c]

    # para cada local, lo que se envía debe ser igual a su requerimiento
    for l in locales:
        problem += pulp.lpSum(X[(c,l)] for c in cds) == requerimientos_local[l]

    return problem, X


### Solución del Problema

In [17]:
problema, x = get_problem(locales, requerimientos_local, cds, disponibilidad_cd, costos)

In [18]:
problema.solve(pulp.PULP_CBC_CMD(msg=True))

1

In [19]:
# Imprimir resultados
print(f"Estado: {pulp.LpStatus[problema.status]}")
print(f"Costo total: {pulp.value(problema.objective)}")

Estado: Optimal
Costo total: 15095.0


In [20]:
lista_cds=[]
lista_locales=[]
lista_q =[]
for c in cds:
    for l in locales:
        lista_cds.append(c)
        lista_locales.append(l)
        lista_q.append(x[(c, l)].varValue) 

In [21]:
output = pd.DataFrame({"CD":lista_cds
                       ,"Local":lista_locales
                       ,"Cantidad a Enviar": lista_q}
                     )

In [22]:
output["costo unitario"] = [(c,l) for c in cds for l in locales] 

In [23]:
output["costo unitario"] = output["costo unitario"].map(costos)

In [24]:
output["costo"] = output["costo unitario"] * output["Cantidad a Enviar"]

In [25]:
output

,CD,Local,Cantidad a Enviar,costo unitario,costo
0,1,a,0.0,46,0.0
1,1,b,20.0,51,1020.0
2,1,c,0.0,65,0.0
3,1,d,55.0,86,4730.0
4,2,a,80.0,35,2800.0
5,2,b,45.0,41,1845.0
6,2,c,0.0,69,0.0
7,2,d,0.0,79,0.0
8,3,a,0.0,99,0.0
9,3,b,0.0,68,0.0
